In [29]:
import os 


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from matplotlib import pylab as plt
import matplotlib.dates as mdates
plt.rcParams['figure.figsize'] = (15.0, 8.0)
import seaborn as sns
from sklearn.model_selection import train_test_split

In [8]:
train = pd.read_csv('/Users/QUAN/Documents/Machine Learning/salesData.csv')
test = pd.read_csv('/Users/QUAN/Documents/Machine Learning/sales_test.csv')
items = pd.read_csv('/Users/QUAN/Documents/Machine Learning/items.csv')
#item_categories = pd.read_csv('/Users/QUAN/Documents/Machine Learning/item_categories.csv')

In [11]:
train_clean = train.drop(labels = ['date', 'item_price','Unnamed: 0'], axis = 1)
train_clean.head()

,date_block_num,shop_id,item_id,item_cnt_day
0,0,59,22154,1
1,0,25,2552,1
2,0,25,2552,-1
3,0,25,2554,1
4,0,25,2555,1


In [12]:
# change the item count per day to item count per month by using group
train_clean = train_clean.groupby(["item_id","shop_id","date_block_num"]).sum().reset_index()
train_clean = train_clean.rename(index=str, columns = {"item_cnt_day":"item_cnt_month"})
train_clean = train_clean[["item_id","shop_id","date_block_num","item_cnt_month"]]
train_clean

,item_id,shop_id,date_block_num,item_cnt_month
0,0,54,20,1
1,1,55,15,2
2,1,55,18,1
3,1,55,19,1
4,1,55,20,1
...,...,...,...,...
1609119,22168,12,8,1
1609120,22168,16,1,1
1609121,22168,42,1,1
1609122,22168,43,2,1


In [30]:
train_clean = train.groupby(["item_id","shop_id","date_block_num"]).sum().reset_index()
train_clean = train_clean.rename(index=str, columns = {"item_cnt_day":"item_cnt_month"})
train_clean = train_clean[["item_id","shop_id","date_block_num","item_cnt_month"]]
train_clean



,item_id,shop_id,date_block_num,item_cnt_month
0,0,54,20,1
1,1,55,15,2
2,1,55,18,1
3,1,55,19,1
4,1,55,20,1
...,...,...,...,...
1609119,22168,12,8,1
1609120,22168,16,1,1
1609121,22168,42,1,1
1609122,22168,43,2,1


In [31]:


predictors = train_clean.drop(["item_cnt_month"],axis = 1)
response = train_clean["item_cnt_month"]
x_train_value, x_value, y_train_value, y_value = train_test_split(predictors, response, test_size = 0.25, random_state = 0)



In [ ]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(15, input_shape=(1,8)))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [66]:
# Reshape the data between -1 and 1 and to 3D
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = StandardScaler()
scaler = MinMaxScaler(feature_range=(-1, 1))
x_train_scaled = scaler.fit_transform(x_train_value)
x_valid_scaled = scaler.fit_transform(x_value)

In [67]:
x_train_reshaped = x_train_scaled.reshape((x_train_scaled.shape[0], 1, x_train_scaled.shape[1]))
x_val_resaped = x_valid_scaled.reshape((x_valid_scaled.shape[0], 1, x_valid_scaled.shape[1]))

In [68]:
history = model_lstm.fit(x_train_reshaped, y_train_value, validation_data=(x_val_resaped, y_value),epochs=70, batch_size=12, verbose=2, shuffle=False)
y_pre = model_lstm.predict(x_val_resaped)

In [70]:
from sklearn.metrics import mean_squared_error
from numpy import sqrt
rmse = sqrt(mean_squared_error(y_value,y_pre))
print('Val RMSE: %.3f' % rmse)